# Semantic Dictionary Segmentation

This notebook aims to test the technique of using a dictionary of semantic terms in order to segment the environment into compartments. These terms will be aggregated from online resources for descriptions of key outdoor objects. 

For each term, we will use a set of positive queries to understand the similarity to the meshed environment. These positive queries will be contrasted with negative queries (obtained from the vocabulary terms of the other items of the dictionary).

For example:

- 'tree': ['green', 'leaves', 'bark']
- 'ground': ['dirt', 'rocks', 'floor']

These terms would then be contrasted against each other.